<a href="https://colab.research.google.com/github/LenkaMikova/NoisMaps/blob/main/DP_LMI_dataDown_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DIPLOMOVÁ PRÁCE
## Hluková mapa odvozená ze satelitních dat

Část zabývající se stažení vstupních dat.

# 0. Technické

Pro zjištěný, jaký procesor GPU byl přiřazen v rámci prémiových GPU.
Poznámka: *Pokud spuštění vede k výsledku „Not connected to a GPU“ , může se běhové prostředí změnit tak, že se v nabídce přejde na Běh > Změnit typ běhového prostředí, povolíte akcelerátor GPU a poté buňku s kódem spustit znovu.*

## Rychlejší GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


# 1. Úvodní




**Raději nejprve odpojit Google Disk napravo v nabídkce v liště. Teprve potom instalovat balíčky a importovat.**

## 1.1 Instalace a importy

### 1.1.1 Instalace balíčků

[Open EO](https://open-eo.github.io/openeo-python-client/) - back-ends to process remote sensing and Earth observation data. It provides a Pythonic interface for the openEO API, supporting data/process discovery, process graph building, batch job management and much more.

In [ ]:
# musi se instalovat pokazde!!!!
!pip install openeo

[Rasterio](https://pypi.org/project/rasterio/) - reads and writes geospatial raster data.

In [ ]:
# musi se instalovat pokazde!!!!
!pip install rasterio

[Geemap](https://geemap.org/) - interactive geospatial analysis and visualization with Google Earth Engine

In [ ]:
# nainstalovat jen jednou
!pip install -U geemap

[Earth Engine Python API](https://github.com/google/earthengine-api/blob/master/README.md) - client libraries for calling the Google Earth Engine AP

In [ ]:
# nainstalovat jen jednou
!pip install earthengine-api
# Pri problemech s instalaci se doporucuje odinstalovat a znovu nainstalovat
#!pip uninstall earthengine-api -y

[iJSON](https://pypi.org/project/ijson/) - Ijson is an iterative JSON parser with standard Python iterator interfaces

In [ ]:
# nainstalovat jen jednou
!pip install ijson

[Matplotlib](https://pypi.org/project/matplotlib/) - comprehensive library for creating static, animated, and interactive visualizations in Python

In [ ]:
!pip install matplotlib

[GDAL](https://pypi.org/project/GDAL/) - a number of tools for programming and manipulating the GDAL Geospatial Data Abstraction Library

In [ ]:
!pip install gdal

In [ ]:
!apt-get install -y gdal-bin

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
gdal-bin is already the newest version (3.6.4+dfsg-1~jammy0).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!gdal_translate --version

GDAL 3.6.4, released 2023/04/17


In [ ]:
!pip install geopandas

### 1.1.2 Import balíčků

raterio

In [ ]:
import rasterio

In [ ]:
from rasterio.sample import sample_gen

[os](https://docs.python.org/3/library/os.html) - using operating system dependent functionality

In [ ]:
import os

[Earth Engine Python API](https://github.com/google/earthengine-api/blob/master/README.md) - client libraries for calling the Google Earth Engine AP

In [ ]:
import ee

[Geemap](https://geemap.org/) - interactive geospatial analysis and visualization with Google Earth Engine. [Geemap Core](https://geemap.org/core/) - generic Map interface and lightweight implementation

In [ ]:
import geemap

In [ ]:
import geemap.core as geemapCore

[Open EO](https://open-eo.github.io/openeo-python-client/) - back-ends to process remote sensing and Earth observation data. It provides a Pythonic interface for the openEO API, supporting data/process discovery, process graph building, batch job management and much more.

In [ ]:
import openeo

[geopandas](https://pypi.org/project/geopandas/) - add support for geographic data to pandas objects

In [ ]:
import geopandas as gpd

[pandas](https://pypi.org/project/pandas/) - fast, flexible, and expressive data structures designed to make working with "relational" or "labeled" data both easy and intuitive

In [ ]:
import pandas as pd

[time](https://docs.python.org/3/library/time.html) - various time-related functions

In [ ]:
import time

[DateTime](https://pypi.org/project/DateTime/) - DateTime data type, as known from Zope.

In [ ]:
import datetime

[Matplotlib](https://pypi.org/project/matplotlib/) - comprehensive library for creating static, animated, and interactive visualizations in Python

In [ ]:
import matplotlib.pyplot as plt

[NumPy](https://pypi.org/project/numpy/) - fundamental package for scientific computing with Python

In [ ]:
import numpy as np

[shutil](https://docs.python.org/3/library/shutil.html) - offers a number of high-level operations on files and collections of files. In particular, functions are provided which support file copying and removal

In [ ]:
import shutil

[requests](https://pypi.org/project/requests/) - simple, yet elegant, HTTP library

In [ ]:
import requests

[Fiona](https://pypi.org/project/fiona/) - simple feature data to and from GIS formats like GeoPackage and Shapefile.

In [ ]:
#import fiona

[Shapely](https://shapely.readthedocs.io/en/stable/) - Shapely is a BSD-licensed Python package for manipulation and analysis of planar geometric objects.

In [ ]:
from shapely.geometry import box

## 1.2 Připojení externích služeb

### 1.2.1 Google Drive

#### Odpojení disku - v případě problémů s diskem

V případě, že bude problém s připojeným Google Disk je třeba takto odpojit případně v Soubor přimov Colam zrušit a znovu spustit připojení.

In [ ]:
#from google.colab import drive
#drive.flush_and_unmount()
#print("Drive unmounted")

#### Připojení disku

In [ ]:
from google.colab import drive
drive.mount('/content/drive4/')
print("Drive mounted")

Drive already mounted at /content/drive4/; to attempt to forcibly remount, call drive.mount("/content/drive4/", force_remount=True).
Drive mounted


#### Nastavení výstupního adresáře

In [ ]:
shp_dir = '/content/drive4/MyDrive/DP'
points_path = os.path.join(shp_dir, "SHM_Body1400_1000m.shp'")

out_path = '/content/drive4/MyDrive/DP/GEE_output'
if not os.path.exists(out_path):
    os.makedirs(out_path)

### 1.2.2 Google Earth Engine

In [ ]:
ee.Authenticate()
print("Authenticate successful")

Authenticate successful


In [ ]:
# V pripade problemu je nutno v GEE přímo v účtu registrovat projekt (vpravo nahore)
try:
    ee.Initialize(project='lenkamikova256')
    print("Initialization successful")
except ee.EEException as e:
    print("Initialization failed: ", e)

Initialization successful


# 2. Satelitní data

## 2.1 Vstupní data a parametry

### 2.1.1 Nastavení vstupních parametrů

In [ ]:
startDate = '2022-11-01' # pocatecni datum sberu dat
endDate = '2022-11-30'  # konecne datu sberu dat
center = [49.926472, 14.861755]
zoom = 10

### 2.1.2 Nastavení vstupních dat

Vstupní data ze ShapeFile a kontrola geometrie (uložená v GoogleDisk).

#### Oblast hluku

In [ ]:
# Test, zda soubor existuje a kontrola ostatních souborů
required_files = ['oblast_hlukData.shp', 'oblast_hlukData.shx', 'oblast_hlukData.dbf', 'oblast_hlukData.prj']
missing_files = [f for f in required_files if not os.path.exists(os.path.join(shp_dir, f))]

if missing_files:
    raise FileNotFoundError(f"Missing required shapefile components: {missing_files}")

shp_pathO = os.path.join(shp_dir, 'oblast_hlukData.shp')

try:
    gdfO = gpd.read_file(shp_pathO)
    print("File oblast_hlukData.shp file loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Error loading SHP file: {e}")

File oblast_hlukData.shp file loaded successfully.


In [ ]:
try:
    gdfO = gdfO.to_crs(epsg=4326)
    print("Point layer converted to WGS84 (EPSG:4326)")
except Exception as e:
    print(f"Error loading shapefile: {e}")

Point layer converted to WGS84 (EPSG:4326)


In [ ]:
if gdfO.empty:
    raise ValueError("Geodataframe is empty. Check the input shapefile.")

polygonOblast = gdfO.geometry.iloc[0]
print("Geometry oblast_hlukData.shp loaded successfully.")

Geometry oblast_hlukData.shp loaded successfully.


In [ ]:
# Pred prevodem je treba provest autentifikaci a inicializace GEE
try:
    oblast = geemap.gdf_to_ee(gdfO)
    print("Geometry oblast converted to Earth Engine format successfully.")
except Exception as e:
    raise RuntimeError(f"Error converting geometry oblast to Earth Engine format: {e}")

Geometry oblast converted to Earth Engine format successfully.


#### Česká republika

In [ ]:
# Jako u oblast_HlukData
required_files = ['STATY_P.shp', 'STATY_P.shx', 'STATY_P.dbf', 'STATY_P.prj']
missing_files = [f for f in required_files if not os.path.exists(os.path.join(shp_dir, f))]

if missing_files:
    raise FileNotFoundError(f"Missing required shapefile components: {missing_files}")

shp_pathCR = os.path.join(shp_dir, 'STATY_P.shp')

try:
    gdfCR = gpd.read_file(shp_pathCR)
    print("File STATY_P.shp file loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Error loading SHP file: {e}")

File STATY_P.shp file loaded successfully.


In [ ]:
try:
    gdfCR = gdfCR.to_crs(epsg=4326)
    print("Point layer converted to WGS84 (EPSG:4326)")
except Exception as e:
    print(f"Error loading shapefile: {e}")

Point layer converted to WGS84 (EPSG:4326)


In [ ]:
if gdfCR.empty:
    raise ValueError("Geodataframe is empty. Check the input shapefile.")

polygonCR = gdfCR.geometry.iloc[0]
print("Geometry STATY_P.shp loaded successfully.")

Geometry STATY_P.shp loaded successfully.


In [ ]:
try:
    CR = geemap.gdf_to_ee(gdfCR)
    print("Geometry CR converted to Earth Engine format successfully.")
except Exception as e:
    raise RuntimeError(f"Error converting geometry CR to Earth Engine format: {e}")

Geometry CR converted to Earth Engine format successfully.


#### Vygenerované body pro oblast hluku

In [ ]:
# Jako u predeslych dvou shp
required_files = ['SHM_Body_1000m_SJ.shp', 'SHM_Body_1000m_SJ.shx', 'SHM_Body_1000m_SJ.dbf', 'SHM_Body_1000m_SJ.prj']
missing_files = [f for f in required_files if not os.path.exists(os.path.join(shp_dir, f))]

if missing_files:
    raise FileNotFoundError(f"Missing required shapefile components: {missing_files}")

shp_pathBody = os.path.join(shp_dir, 'SHM_Body_1000m_SJ.shp')

try:
    gdfBody = gpd.read_file(shp_pathBody)
    print("File SHM_Body_1000m_SJ.shp file loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Error loading SHP file: {e}")

File SHM_Body_1000m_SJ.shp file loaded successfully.


In [ ]:
try:
    gdfBody = gdfBody.to_crs(epsg=4326)
    print("Point layer converted to WGS84 (EPSG:4326)")
except Exception as e:
    print(f"Error loading shapefile: {e}")

Point layer converted to WGS84 (EPSG:4326)


In [ ]:
if gdfBody.empty:
    raise ValueError("Geodataframe is empty. Check the input shapefile.")

Body1400 = gdfBody.geometry.iloc[0]
print("Geometry SHM_Body_1000m_SJ.shp loaded successfully.")

Geometry SHM_Body_1000m_SJ.shp loaded successfully.


In [ ]:
try:
    Body1400_ee = geemap.gdf_to_ee(gdfBody)
    print("Geometry SHM_Body_1000m_SJ converted to Earth Engine format successfully.")
except Exception as e:
    raise RuntimeError(f"Error converting geometry SHM_Body_1000m_SJ to Earth Engine format: {e}")

Geometry SHM_Body_1000m_SJ converted to Earth Engine format successfully.


In [ ]:
print(gdfBody.columns)

Index(['ID', 'LDEN', 'noiseLevel', 'MERGE_SRC', 'Shape_Leng', 'geometry'], dtype='object')


In [ ]:
print(gdfBody)

        ID   LDEN         noiseLevel   MERGE_SRC    Shape_Leng  \
0        1  60.00           Lden5559  aglomerace  2.618360e+06   
1        2  65.00           Lden6064     silnice  1.327160e+06   
2        3  80.00  LdenGreaterThan75     silnice  7.238570e+05   
3        4  70.00           Lden6569   zeleznice  8.856371e+05   
4        5  65.00           Lden6064   zeleznice  1.107023e+06   
...    ...    ...                ...         ...           ...   
1395  1396  75.00           Lden7074   zeleznice  1.043129e+06   
1396  1397  55.00           Lden5054  aglomerace  9.099600e+05   
1397  1398  55.00           Lden5054     silnice  7.512108e+06   
1398  1399  60.00           Lden5559     silnice  1.537878e+06   
1399  1400  55.00           Lden5054     silnice  2.570603e+06   

                       geometry  
0     POINT (16.62017 49.12896)  
1     POINT (13.33214 49.67493)  
2     POINT (13.20077 49.70329)  
3     POINT (16.48606 49.78299)  
4     POINT (16.57605 49.89006)  
...

#### Rozdělení polygonu a velikost mřížky

Rozdělení polygonu na menší dlaždice z důvodu vysoké výpočetní náročnosti při volbě celého území.

In [ ]:
def split_polygon(polygon, rows, cols):
    minx, miny, maxx, maxy = polygon.bounds
    width = (maxx - minx) / cols
    height = (maxy - miny) / rows
    tiles = []
    for i in range(cols):
        for j in range(rows):
            minx_tile = minx + i * width
            miny_tile = miny + j * height
            maxx_tile = minx + (i + 1) * width
            maxy_tile = miny + (j + 1) * height
            tile = box(minx_tile, miny_tile, maxx_tile, maxy_tile)
            tiles.append(tile)
    return tiles

Rozdělení oblasti na mřížku

In [ ]:
mrizka = 32 # zvolit vhodne
tiles = split_polygon(polygonOblast, mrizka,mrizka)
print(f"Tiles is set up to {mrizka} x {mrizka} ")

Tiles is set up to 32 x 32 


Vytvoření GeoDataFrame pro dlaždice

In [ ]:
tiles_gdf = gpd.GeoDataFrame({'geometry': tiles}, crs=gdfO.crs)
print("GeoData Frame created successfully.")

GeoData Frame created successfully.


Validace pro kontrolu a opravu platnosti geometrie (kontrola- prázdné a neplatné geometrie, odstranění prázdné geometrie a oprava neplatné).

In [ ]:
def validate_geometry(gdf):
    if 'geometry' not in gdf.columns:
        raise ValueError("GeoDataFrame does not contain 'geometry' column")

    if gdf['geometry'].isnull().any():
        print("GeoDataFrame contains empty geometries. They will be removed.")
        gdf = gdf.dropna(subset=['geometry'])

    invalid_geom = gdf[~gdf.is_valid]
    if not invalid_geom.empty:
        print(f"GeoDataFrame contains {len(invalid_geom)} invalid geometry. Attempting to repair...")

        gdf['geometry'] = gdf['geometry'].buffer(0) # oprava neplatne geometrie

        invalid_geom_after = gdf[~gdf.is_valid] # opetovna kontrola
        if not invalid_geom_after.empty:
            print(f"{len(invalid_geom_after)} geometries still remain invalid.")
        else:
            print("All invalid geometries have been repaired successfully.")
    else:
        print("All geometries in GeoData Frame are valid.")

    valid_gdf = gdf[gdf.is_valid]
    return valid_gdf

In [ ]:
tiles_gdf = validate_geometry(tiles_gdf)

All geometries in GeoData Frame are valid.


Převod dlaždic pro GEE

In [ ]:
tiles_ee = [geemap.geopandas_to_ee(tiles_gdf) for tile_gdf in tiles_gdf.geometry]
print("Tiles converted for GEE successfully.")

Tiles converted for GEE successfully.


### 2.1.3 Barevné škály pro indexy

In [ ]:
color_schemes = {
    "NDVI": {
        "min": 0.6,  # upraveno podle rozsahu dat
        "max": 1.0,
        "palette": [
            "#d73027",  # Red (low vegetation or negative values)
            "#fc8d59",  # Light red
            "#fee08b",  # Yellow
            "#d9ef8b",  # Light green
            "#91cf60",  # Green
            "#1a9850"   # Dark green (dense vegetation)
        ]
    },
    "NDBI": {
        "min": -0.3, # upraveno podle rozsahu dat
        "max": 0.0, # upraveno podle rozsahu dat
        "palette": [
            "#2c7bb6",  # Blue (low built-up or negative values)
            "#abd9e9",  # Light blue
            "#ffffbf",  # Yellowish
            "#fdae61",  # Orange
            "#d7191c"   # Red (high built-up areas)
        ]
    },
    "MNDWI": {
        "min": -0.5, # upraveno podle rozsahu dat
        "max": 0.1, # upraveno podle rozsahu dat
        "palette": [
            "#fddbc7",  # Light pink (low water presence or negative values)
            "#ef8a62",  # Orange
            "#999999",  # Neutral
            "#67a9cf",  # Light blue
            "#2166ac"   # Dark blue (high water presence)
        ]
    },
    "ISA": {  # Impervious Surface Area
        "min": -0.9, # upraveno podle rozsahu dat
        "max": -0.3, # upraveno podle rozsahu dat
        "palette": [
            "#ffffff",  # White (low impervious surface)
            "#d9d9d9",  # Light gray
            "#969696",  # Gray
            "#525252",  # Dark gray
            "#000000"   # Black (high impervious surface)
        ]
    },
    "UI": {  # Urban Index
        "min": -0.3, # upraveno podle rozsahu dat
        "max": 0.3, # upraveno podle rozsahu dat
        "palette": [
            "#f7f7f7",  # Light gray (neutral or low urban intensity)
            "#cccccc",  # Gray
            "#969696",  # Medium gray
            "#525252",  # Dark gray
            "#000000"   # Black (high urban intensity)
        ]
    },
    "TIR_K": {
        "min": 299.0,  # min hodnota Kelviny
        "max": 320.0,  # max hodnota Kelviny
        "palette": [
            "#0000ff",  # Blue (low temperature)
            "#00ffff",  # Cyan
            "#00ff00",  # Green
            "#ffff00",  # Yellow
            "#ff7f00",  # Orange
            "#ff0000"   # Red (high temperature)
        ]
    },
    "TIR_C": {
        "min": -3.0,  # min hodnota Celsia
        "max": 50.0,  # max hodnota Celsia
        "palette": [
            "#0000ff",  # Blue (low temperature)
            "#00ffff",  # Cyan
            "#00ff00",  # Green
            "#ffff00",  # Yellow
            "#ff7f00",  # Orange
            "#ff0000"   # Red (high temperature)
        ]
    }
}


## 2.2 LANDSAT 8

### 2.2.1 Indexy

In [ ]:
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

In [ ]:
def rename_bands(image):
    return image.select(
        ['SR_B5', 'SR_B6', 'SR_B4', 'SR_B3', 'SR_B2', 'ST_B10'],
        ['NIR', 'SWIR1', 'red', 'green', 'blue', 'TIR1']
    )

NDVI je spektrální index používaný k detekci a kvantifikaci vegetace na povrchu Země.

#### **NDVI** přes GEE

In [ ]:
def calculate_ndvi(image):
    return image.normalizedDifference(['NIR', 'red']).rename('NDVI')


In [ ]:
def calculate_ndvi_tiles(tile):
    landsat_collection_tile = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(apply_scale_factors) \
        .map(rename_bands) \
        .filterBounds(tile.geometry()) \
        .map(calculate_ndvi)

    ndvi_median = landsat_collection_tile.median().rename('NDVI_Median').clip(tile.geometry())
    ndvi_max = landsat_collection_tile.max().rename('NDVI_Max').clip(tile.geometry())

    return ndvi_median, ndvi_max

ndvi_median_tiles = []
ndvi_max_tiles = []

for tile in tiles_ee:
    median, max_val = calculate_ndvi_tiles(tile)
    ndvi_median_tiles.append(median)
    ndvi_max_tiles.append(max_val)

print("NDVI Median and Max calculated and stored in separate lists.")


NDVI Median and Max calculated and stored in separate lists.


In [ ]:
ndvi_median_collection = ee.ImageCollection(ndvi_median_tiles)
ndvi_max_collection = ee.ImageCollection(ndvi_max_tiles)

In [ ]:
ndvi_median_mosaic = ndvi_median_collection.mosaic().rename('NDVI_Median')
print("Mosaic NDVI Median created.")

ndvi_max_mosaic = ndvi_max_collection.mosaic().rename('NDVI_Max')
print("Mosaic NDVI Max created.")

Mosaic NDVI Median created.
Mosaic NDVI Max created.


In [ ]:
statsNDVI = ndvi_max_mosaic.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=Body1400_ee.geometry(),
    #scale=30,
    #maxPixels=1e13
)

print("Min a Max hodnoty NDVI:", statsNDVI.getInfo())

Min a Max hodnoty NDVI: {'NDVI_Max_max': 0.7119055390357971, 'NDVI_Max_min': 0.16151879727840424}


Min a max je v rozmezi 0,62 az 0,87. Jsou to hodnoty NDVI pouze ve vybranych bodech. Takze ve vybranych 1400 bodech neni zadna voda.

In [ ]:
NDVI_map = geemap.Map(center=center, zoom=zoom)
NDVI_map.addLayer(ndvi_max_mosaic, color_schemes["NDVI"], 'NDVI max')
NDVI_map.setCenter(14.861755, 49.926472, zoom)
NDVI_map

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **NDBI** přes GEE

NDBI je spektrální index, který se používá k identifikaci zastavěných oblastí, včetně průmyslových a komerčních ploch.

In [ ]:
def calculate_ndbi(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(apply_scale_factors) \
        .map(rename_bands) \
        .filterBounds(tile)

    ndbi = landsat_collection \
        .map(lambda image: image.normalizedDifference(['SWIR1', 'NIR']).rename('NDBI')) \
        .median() \
        .clip(tile)

    return ndbi

In [ ]:
ndbi_tiles = [calculate_ndbi(tile, startDate, endDate) for tile in tiles_ee]
print("NDBI calculated.")

NDBI calculated.


In [ ]:
ndbi_mosaic = ee.ImageCollection(ndbi_tiles).mosaic()

In [ ]:
statsNDBI = ndbi_mosaic.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=Body1400_ee.geometry(),
    #scale=30,
    #maxPixels=1e13
)

print("Min a Max hodnoty NDBI:", statsNDBI.getInfo())

Min a Max hodnoty NDBI: {'NDBI_max': 0.037606194615364075, 'NDBI_min': -0.6455783843994141}


In [ ]:
NDBImap = geemap.Map(center=center, zoom=zoom)
NDBImap.addLayer(ndbi_mosaic, color_schemes["NDBI"], 'NDBI Mosaic')
NDBImap.setCenter(14.861755, 49.926472, zoom)
NDBImap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **MNDWI** přes GEE

MNDWI může být také užitečný pro identifikaci ne-zastavěných a zastavěných oblastí, protože zlepšuje rozdíly mezi vodními a zastavěnými plochami.

In [ ]:
 def calculate_mndwi(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(apply_scale_factors) \
        .map(rename_bands) \
        .filterBounds(tile)

    mndwi = landsat_collection \
        .map(lambda image: image.normalizedDifference(['green', 'SWIR1']).rename('MNDWI')) \
        .median() \
        .clip(tile)

    return mndwi

In [ ]:
mndwi_tiles = [calculate_mndwi(tile, startDate, endDate) for tile in tiles_ee]
print("MNDVW calculated.")

MNDVW calculated.


In [ ]:
mndwi_mosaic = ee.ImageCollection(mndwi_tiles).mosaic()

In [ ]:
statsMNDWI = mndwi_mosaic.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=Body1400_ee.geometry(),
    #scale=30,
    #maxPixels=1e13
)

print("Min a Max hodnoty MNDWI:", statsMNDWI.getInfo())

Min a Max hodnoty MNDWI: {'MNDWI_max': 0.22429035604000092, 'MNDWI_min': -0.505379319190979}


In [ ]:
MNDWImap = geemap.Map(center=center, zoom=zoom)
MNDWImap.addLayer(mndwi_mosaic, color_schemes["MNDWI"], 'MNDWI Mosaic')
MNDWImap.setCenter(14.861755, 49.926472, zoom)
MNDWImap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **UI** přes GEE

Urban Index je další index, který může být použit k detekci urbanizovaných ploch.

In [ ]:
def calculate_ui(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(apply_scale_factors) \
        .map(rename_bands) \
        .filterBounds(tile)

    ui = landsat_collection \
        .map(lambda image: image.expression(
            '((NIR - (red + green)) / (NIR + (red + green)))',
            {
                'NIR': image.select('NIR'),
                'red': image.select('red'),
                'green': image.select('green')
            }).rename('UI')
        ) \
        .median() \
        .clip(tile)

    return ui

In [ ]:
ui_tiles = [calculate_ui(tile, startDate, endDate) for tile in tiles_ee]
print("UI calculated.")

UI calculated.


In [ ]:
ui_mosaic = ee.ImageCollection(ui_tiles).mosaic()

In [ ]:
statsUI = ui_mosaic.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=Body1400_ee.geometry(),
    #scale=30,
    #maxPixels=1e13
)

print("Min a Max hodnoty UI:", statsUI.getInfo())

Min a Max hodnoty UI: {'UI_max': 0.26247037856681227, 'UI_min': -0.19734626236909947}


In [ ]:
UImap = geemap.Map(center=center, zoom=zoom)
UImap.addLayer(ui_mosaic, color_schemes["UI"], 'UI Mosaic')
UImap.setCenter(14.861755, 49.926472, zoom)
UImap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **ISA** přes GEE

ISA index se používá k odhadu nepropustných povrchů, které jsou typické pro
průmyslové a komerční oblasti.

In [ ]:
def calculate_isa(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(apply_scale_factors) \
        .map(rename_bands) \
        .filterBounds(tile.geometry())

    ndvi = landsat_collection.map(calculate_ndvi).median().rename('NDVI').clip(tile.geometry())
    ndbi = calculate_ndbi(tile, startDate, endDate)

    # rozdil NDBI - NDVI
    isa = ndbi.subtract(ndvi).rename('ISA')

    return isa

In [ ]:
isa_tiles = [calculate_isa(tile, startDate, endDate) for tile in tiles_ee]
print("ISA calculated.")

ISA calculated.


In [ ]:
isa_mosaic = ee.ImageCollection(isa_tiles).mosaic()

In [ ]:
statsISA = isa_mosaic.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=Body1400_ee.geometry(),
    #scale=30,
    #maxPixels=1e13
)

print("Min a Max hodnoty ISA:", statsISA.getInfo())

Min a Max hodnoty ISA: {'ISA_max': -0.32491588592529297, 'ISA_min': -1.1839003562927246}


In [ ]:
ISAmap = geemap.Map(center=center, zoom=zoom)
ISAmap.addLayer(isa_mosaic, color_schemes["ISA"], 'ISA Mosaic')
ISAmap.setCenter(14.861755, 49.926472, zoom)
ISAmap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

#### **Thermal data** přes GEE

In [ ]:
def download_tir(tile, startDate, endDate):
    landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
        .filterDate(startDate, endDate) \
        .map(apply_scale_factors) \
        .map(rename_bands) \
        .filterBounds(tile)

    tir = landsat_collection.select('TIR1').max().clip(tile)

    return tir

In [ ]:
tir_tiles = [download_tir(tile, startDate, endDate) for tile in tiles_ee]
tir_mosaic = ee.ImageCollection(tir_tiles).mosaic()

In [ ]:
statsTIR = tir_mosaic.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=Body1400_ee.geometry(),
    #scale=30,
    #maxPixels=1e13
)

print("Min a Max hodnoty TIR:", statsTIR.getInfo())

Min a Max hodnoty TIR: {'TIR1_max': 283.36578422, 'TIR1_min': 263.04565532}


In [ ]:
TIRmap = geemap.Map(center=center, zoom=zoom)
TIRmap.addLayer(tir_mosaic, color_schemes["TIR_K"], 'TIR Mosaic')
TIRmap.setCenter(14.861755, 49.926472, zoom)
TIRmap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

### 2.2.2 Výstupy

Skript kombinuje několik snímků do jedné mozaiky, která představuje sjednocený obraz, kde jsou vybrány nejlepší hodnoty pixelů z překrývajících se snímků.

In [ ]:
ndvi_median_mosaic = ndvi_median_collection.mosaic().rename('NDVI_Median')
print("Mosaic median NDVI created.")
ndvi_max_mosaic = ndvi_max_collection.mosaic().rename('NDVI_Max')
print("Mosaic max NDVI created.")
ndbi_mosaic = ee.ImageCollection(ndbi_tiles).mosaic().rename('NDBI')
print("Mosaic NDBI created.")
mndwi_mosaic = ee.ImageCollection(mndwi_tiles).mosaic().rename('MNDWI')
print("Mosaic MNDWI created.")
ui_mosaic = ee.ImageCollection(ui_tiles).mosaic().rename('UI')
print("Mosaic UI created.")
isa_mosaic = ee.ImageCollection(isa_tiles).mosaic().rename('ISA')
print("Mosaic ISA created.")
tir_mosaic = ee.ImageCollection(tir_tiles).mosaic().rename('TIR_Max')
print("Mosaic TIR created.")

Mosaic median NDVI created.
Mosaic max NDVI created.
Mosaic NDBI created.
Mosaic MNDWI created.
Mosaic UI created.
Mosaic ISA created.
Mosaic TIR created.


In [ ]:
landsat_combined = ee.Image.cat([ndvi_median_mosaic.rename('NDVI_Median'),
                         ndvi_max_mosaic.rename('NDVI_Max'),
                         ndbi_mosaic.rename('NDBI'),
                         mndwi_mosaic.rename('MNDWI'),
                         ui_mosaic.rename('UI'),
                         isa_mosaic.rename('ISA'),
                         tir_mosaic.rename('TIR_Max')])
print("The combination of several images (frames) from different indexes into one multi-band image is complete.")

The combination of several images (frames) from different indexes into one multi-band image is complete.


In [ ]:
landsat_samples = landsat_combined.sampleRegions(
    collection=Body1400_ee,  # Nactena bodova vrstva
    scale=30,
    geometries=True
)
print("Landsat 8 samples extracted.")

Landsat 8 samples extracted.


#### Data z Landsat 8 do bodové vrstvy

In [ ]:
taskL8 = ee.batch.Export.table.toDrive(
    collection=landsat_samples,
    description='Landsat_8_samples_export',
    folder='GEE_output',
    fileNamePrefix='landsat8_samples',
    fileFormat='CSV'
)

taskL8.start()
print('Export task started. Check your Google Drive for the output.')

Export task started. Check your Google Drive for the output.


In [ ]:
landsat8_file = '/content/drive4/MyDrive/DP/GEE_output/landsat8_samples.csv'

In [ ]:
dataL8 = pd.read_csv(landsat8_file)
print(dataL8.head())

  system:index  ID       ISA  LDEN   MERGE_SRC     MNDWI      NDBI  NDVI_Max  \
0          0_0   1 -0.586670  60.0  aglomerace -0.101433 -0.259374  0.709623   
1          1_0   2 -0.231449  65.0     silnice -0.509985  0.080126  0.311795   
2          2_0   3 -0.188383  80.0     silnice -0.349204  0.021855  0.211476   
3          3_0   4 -0.238584  70.0   zeleznice -0.350593  0.025139  0.263723   
4          4_0   5 -1.131270  65.0   zeleznice -0.371166 -0.352987  0.778282   

   NDVI_Median    Shape_Leng     TIR_Max        UI         noiseLevel  \
0     0.327296  2.618360e+06  301.262537  0.015710           Lden5559   
1     0.311575  1.327160e+06  307.346613  0.049354           Lden6064   
2     0.210238  7.238570e+05  323.886411 -0.075291  LdenGreaterThan75   
3     0.263723  8.856371e+05  302.910023 -0.042263           Lden6569   
4     0.778282  1.107023e+06  299.645813  0.488102           Lden6064   

                                                .geo  
0  {"geodesic":false,"typ

## 2.3 MODIS

Nastavení visualizačních parametrů.

In [ ]:
visParams = {
    'min': -10,
    'max': 50,
    'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]
}

Funkce na vzorkování.

In [ ]:
def sample_lst(image):
    return image.sampleRegions(
        collection=Body1400_ee,
        scale=1000,
        geometries=True
    )

#### MODIS A1

[Land Surface Temperature/Emissivity Daily L3 Global 1km - MOD11A1 (Terra Production)](https://lpdaac.usgs.gov/products/mod11a1v006/)

In [ ]:
modis = ee.ImageCollection('MODIS/061/MOD11A1').filter(ee.Filter.date(startDate, endDate))
modLSTday = modis.select('LST_Day_1km')

# prevedeni z Kelvinu na Celsia
modLSTk = modLSTday.map(lambda image: image.multiply(0.02).copyProperties(image, ['system:time_start']))

modisMax = modLSTk.max()

In [ ]:
sampled_lst = modLSTk.map(sample_lst).flatten()

In [ ]:
print('Number of samples:', sampled_lst.size().getInfo())
print('First sample:', sampled_lst.first().getInfo())

Number of samples: 181554
First sample: {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [14.251063252964535, 50.05667589480249]}, 'id': '2022_01_01_8_0', 'properties': {'ID': 9, 'LDEN': '55.00', 'LST_Day_1km': 280.68, 'MERGE_SRC': 'aglomerace', 'Shape_Leng': 8628352.16132, 'noiseLevel': 'Lden5054'}}


Data MODIS A1 do bodové vrstvy

In [ ]:
taskDay = ee.batch.Export.table.toDrive(
    collection=sampled_lst,
    description='MODIS_LST_sample_day',
    folder='GEE_output',
    fileNamePrefix='modis_lst_day',
    fileFormat='CSV'
)
taskDay.start()
print('Export task started. Waiting for completion...')

Export task started. Waiting for completion...


Kontrola stavu

In [ ]:
while taskDay.active():
    print('Waiting for the export to complete...')
    time.sleep(30)

if taskDay.status()['state'] == 'COMPLETED':
    print('Export completed successfully.')
else:
    print('Export failed:', taskDay.status())

Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for 

In [ ]:
modis_file = '/content/drive4/MyDrive/DP/GEE_output/modis_lst_day.csv'
dataMD = pd.read_csv(modis_file)

In [ ]:
dataMD_day = pd.read_csv(modis_file)
print(dataMD_day.head())

      system:index  ID  LDEN  LST_Day_1km   MERGE_SRC    Shape_Leng  \
0   2022_01_01_8_0   9  55.0       280.68  aglomerace  8.628352e+06   
1  2022_01_01_35_0  36  65.0       285.40   zeleznice  9.506742e+05   
2  2022_01_01_41_0  42  55.0       282.42     silnice  7.512108e+06   
3  2022_01_01_74_0  75  55.0       282.56     silnice  1.104389e+06   
4  2022_01_01_75_0  76  55.0       284.34     silnice  3.409484e+06   

  noiseLevel                                               .geo  
0   Lden5054  {"geodesic":false,"type":"Point","coordinates"...  
1   Lden6064  {"geodesic":false,"type":"Point","coordinates"...  
2   Lden5054  {"geodesic":false,"type":"Point","coordinates"...  
3   Lden5054  {"geodesic":false,"type":"Point","coordinates"...  
4   Lden5054  {"geodesic":false,"type":"Point","coordinates"...  


In [ ]:
MODISmap = geemap.Map(center=center, zoom=zoom)
MODISmap.addLayer(modLSTk, visParams, 'MODIS LST Max (K)')
MODISmap.setCenter(14.861755, 49.926472, zoom)
MODISmap

Map(center=[49.926472, 14.861755], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

### MODIS C3

Land Surface Temperature and 3-Band Emissivity Monthly L3 Global 0.05 Deg CMG - MOD21C3.061 (Terra Production)

In [ ]:
modisMonth = ee.ImageCollection('MODIS/061/MOD21C3').filter(ee.Filter.date
 (startDate, endDate))
modLSTmonth = modisMonth.select('LST_Day')

# prevedeni z Kelvinu na Celsia
modLSTc2 = modLSTmonth.map(lambda image: image.multiply(0.02).subtract(273.15)
                        .copyProperties(image, ['system:time_start']))

modisMean2 = modLSTc2.mean()

In [ ]:
sampled_lst2 = modLSTc2.map(sample_lst).flatten()

In [ ]:
print('Number of samples:', sampled_lst2.size().getInfo())
print('First sample:', sampled_lst2.first().getInfo())

Number of samples: 16800
First sample: {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [16.61875781166023, 49.131146148982374]}, 'id': '2022_01_01_0_0', 'properties': {'ID': 1, 'LDEN': '60.00', 'LST_Day': -267.61159999999995, 'MERGE_SRC': 'aglomerace', 'Shape_Leng': 2618360, 'noiseLevel': 'Lden5559'}}


In [ ]:
print(Body1400_ee.size().getInfo())  # pocet prvku v kolekci

1400


Data z MODIS do bodové vrstvy

In [ ]:
taskMonth = ee.batch.Export.table.toDrive(
    collection=sampled_lst2,
    description='MODIS_LST_sample_month',
    folder='GEE_output',
    fileNamePrefix='modis_lst_Month',
    fileFormat='CSV'
)
taskMonth.start()
print('Export task started. Waiting for completion...')

Export task started. Waiting for completion...


Kontrola stavu

In [ ]:
while taskMonth.active():
    print('Waiting for the export to complete...')
    time.sleep(30)

if taskMonth.status()['state'] == 'COMPLETED':
    print('Export completed successfully.')
else:
    print('Export failed:', taskMonth.status())

Waiting for the export to complete...
Export completed successfully.


In [ ]:
modis_file2 = '/content/drive4/MyDrive/DP/GEE_output/modis_lst_Month.csv'

In [ ]:
dataMD_month = pd.read_csv(modis_file2)
print(dataMD_month.head())

     system:index  ID  LDEN   LST_Day   MERGE_SRC    Shape_Leng  \
0  2022_01_01_0_0   1  60.0 -267.6116  aglomerace  2.618360e+06   
1  2022_01_01_1_0   2  65.0 -267.7680     silnice  1.327160e+06   
2  2022_01_01_2_0   3  80.0 -267.6512     silnice  7.238570e+05   
3  2022_01_01_3_0   4  70.0 -267.6924   zeleznice  8.856371e+05   
4  2022_01_01_4_0   5  65.0 -267.6488   zeleznice  1.107023e+06   

          noiseLevel                                               .geo  
0           Lden5559  {"geodesic":false,"type":"Point","coordinates"...  
1           Lden6064  {"geodesic":false,"type":"Point","coordinates"...  
2  LdenGreaterThan75  {"geodesic":false,"type":"Point","coordinates"...  
3           Lden6569  {"geodesic":false,"type":"Point","coordinates"...  
4           Lden6064  {"geodesic":false,"type":"Point","coordinates"...  


In [ ]:
dataMD_month.columns

Index(['system:index', 'ID', 'LDEN', 'LST_Day', 'MERGE_SRC', 'Shape_Leng',
       'noiseLevel', '.geo'],
      dtype='object')

## 2.4 SENTINEL 2

*Lze stahovat jak přes Sentinel Hub tak přes GEE.*

Funkce na přejmenování bandů pro Sentinel 2

In [ ]:
def rename_bands(image):
    return image.select(
        ['B8', 'B11', 'B4', 'B3', 'B2'],  # NIR, SWIR1, Red, Green, Blue
        ['NIR', 'SWIR1', 'red', 'green', 'blue']
    )

### 2.4.1 Indexy

Řada S2_SR není přesná - je tam jiný přepočet. Nutno používat S2_SR_HARMONIZED.

#### **NDVI** přes GEE

In [ ]:
def calculate_s2_ndvi(tile, startDate, endDate):
    sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    ndvi = sentinel_collection \
        .map(lambda image: image.normalizedDifference(['NIR', 'red']).rename('S2_NDVI')) \
        .median() \
        .clip(tile)

    return ndvi

#### **NDBI** přes GEE

In [ ]:
def calculate_s2_ndbi(tile, startDate, endDate):
    sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    ndbi = sentinel_collection \
        .map(lambda image: image.normalizedDifference(['SWIR1', 'NIR']).rename('S2_NDBI')) \
        .median() \
        .clip(tile)

    return ndbi


#### **MNDWI** přes GEE

In [ ]:
def calculate_s2_mndwi(tile, startDate, endDate):
    sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    mndwi = sentinel_collection \
        .map(lambda image: image.normalizedDifference(['green', 'SWIR1']).rename('S2_MNDWI')) \
        .median() \
        .clip(tile)

    return mndwi


#### **UI** přes GEE

In [ ]:
def calculate_s2_ui(tile, startDate, endDate):
    sentinel_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate(startDate, endDate) \
        .map(rename_bands) \
        .filterBounds(tile)

    ui = sentinel_collection \
        .map(lambda image: image.expression(
            '((NIR - (Red + Green)) / (NIR + (Red + Green)))',
            {
                'NIR': image.select('NIR'),
                'Red': image.select('red'),
                'Green': image.select('green')
            }).rename('S2_UI')
        ) \
        .median() \
        .clip(tile)

    return ui


#### **ISA** přes GEE

In [ ]:
def calculate_s2_isa(tile, startDate, endDate):
    # Výpočet NDVI a NDBI
    ndvi = calculate_s2_ndvi(tile, startDate, endDate)
    ndbi = calculate_s2_ndbi(tile, startDate, endDate)

    # Výpočet ISA (rozdíl NDBI - NDVI)
    isa = ndbi.subtract(ndvi).rename('S2_ISA')

    return isa


### 2.4.2 Výstupy

In [ ]:
ndvi_tile = calculate_s2_ndvi(Body1400_ee, startDate, endDate)
ndbi_tile = calculate_s2_ndbi(Body1400_ee, startDate, endDate)
mndwi_tile = calculate_s2_mndwi(Body1400_ee, startDate, endDate)
ui_tile = calculate_s2_ui(Body1400_ee, startDate, endDate)
isa_tile = calculate_s2_isa(Body1400_ee, startDate, endDate)

In [ ]:
S2_combined = ee.Image.cat([ndvi_tile, ndbi_tile, mndwi_tile, ui_tile, isa_tile])
print("Sentinel-2 indices combined into a multi-band image.")

Sentinel-2 indices combined into a multi-band image.


In [ ]:
# vzorkova
s2_samples = S2_combined.sampleRegions(
    collection=Body1400_ee,
    scale=10,
    geometries=True
)

#### Data ze Sentinel 2 do bodové vrstvy

In [ ]:
taskS2 = ee.batch.Export.table.toDrive(
    collection=s2_samples,
    description='Sentinel_2_sample_export',
    folder='GEE_output',
    fileNamePrefix='sentinel2_samples',
    fileFormat='CSV'
)

taskS2.start()
print('Export task for Sentinel-2 indices started.')

Export task for Sentinel-2 indices started.


In [ ]:
while taskS2.active():
    print('Waiting for the export to complete...')
    time.sleep(30)

if taskS2.status()['state'] == 'COMPLETED':
    print('Export completed successfully.')
else:
    print('Export failed:', taskS2.status())

Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Waiting for the export to complete...
Export completed successfully.


In [ ]:
sentinel2_file = '/content/drive4/MyDrive/DP/GEE_output/sentinel2_samples.csv'

In [ ]:
dataS2 = pd.read_csv(sentinel2_file)
print(dataS2.head())

  system:index  ID  LDEN   MERGE_SRC    S2_ISA  S2_MNDWI   S2_NDBI   S2_NDVI  \
0          0_0   1  60.0  aglomerace -0.286460  0.018242 -0.193990  0.092470   
1          1_0   2  65.0     silnice -0.409762 -0.037070 -0.223850  0.185912   
2          2_0   3  80.0     silnice -0.372272 -0.061164 -0.213619  0.158653   
3          3_0   4  70.0   zeleznice -0.319280  0.078823 -0.239279  0.080001   
4          4_0   5  65.0   zeleznice -0.329010  0.121370 -0.251552  0.077458   

      S2_UI    Shape_Leng         noiseLevel  \
0 -0.250048  2.618360e+06           Lden5559   
1 -0.159855  1.327160e+06           Lden6064   
2 -0.150131  7.238570e+05  LdenGreaterThan75   
3 -0.261980  8.856371e+05           Lden6569   
4 -0.261939  1.107023e+06           Lden6064   

                                                .geo  
0  {"geodesic":false,"type":"Point","coordinates"...  
1  {"geodesic":false,"type":"Point","coordinates"...  
2  {"geodesic":false,"type":"Point","coordinates"...  
3  {"geode

## 2.5 SENTINEL 5P

#### Obecné

*Jelikož kolekce ze Sentinel Hub neobsahuje Sentinel 5P, tak je třeba data stahovat prostřednictvím GEE.*

Kontrola kolekce na počet snímků - pokud se zdá, že relace trvá moc krátce. Ověří se, zda v daném období jsou nějaká data.

In [ ]:
# Kontrola velikosti kolekce
def check_collection_size(collection_name):
    collection = ee.ImageCollection(collection_name).filterDate(startDate, endDate)
    size = collection.size().getInfo()  # pocet v kolekci
    print(f"{collection_name} collection size: {size}")
    return collection

In [ ]:
try:
    # Vytvoření a kontrola samostatných obrazů pro každý plyn
    CO = check_collection_size('COPERNICUS/S5P/NRTI/L3_CO').mean().rename('CO')
    HCHO = check_collection_size('COPERNICUS/S5P/NRTI/L3_HCHO').mean().rename('HCHO')
    NO2 = check_collection_size('COPERNICUS/S5P/NRTI/L3_NO2').mean().rename('NO2')
    SO2 = check_collection_size('COPERNICUS/S5P/NRTI/L3_SO2').mean().rename('SO2')
    CH4 = check_collection_size('COPERNICUS/S5P/OFFL/L3_CH4').mean().rename('CH4')

    # Samostatné pásma pro O3
    O3 = check_collection_size('COPERNICUS/S5P/NRTI/L3_O3').select("O3").mean().rename('O3')
    O3_TCL = check_collection_size('COPERNICUS/S5P/NRTI/L3_O3').select("O3_TCL").mean().rename('O3_TCL')

    print("Sentinel 5P layers prepared.")

except Exception as e:
    print(f"Error loading or processing Sentinel-5P ImageCollection: {e}")

COPERNICUS/S5P/NRTI/L3_CO collection size: 65394
COPERNICUS/S5P/NRTI/L3_HCHO collection size: 56401
COPERNICUS/S5P/NRTI/L3_NO2 collection size: 64268
COPERNICUS/S5P/NRTI/L3_SO2 collection size: 45488
COPERNICUS/S5P/OFFL/L3_CH4 collection size: 4653
COPERNICUS/S5P/NRTI/L3_O3 collection size: 72732
COPERNICUS/S5P/NRTI/L3_O3 collection size: 72732
Sentinel 5P layers prepared.


Data jsou průměry za vybrané období.

In [ ]:
try:
    CO = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_CO').filterDate(startDate, endDate).select("CO_column_number_density").mean().rename('CO')
    HCHO = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_HCHO').filterDate(startDate, endDate).select("tropospheric_HCHO_column_number_density").mean().rename('HCHO')
    NO2 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_NO2').filterDate(startDate, endDate).select("tropospheric_NO2_column_number_density").mean().rename('NO2')
    SO2 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2').filterDate(startDate, endDate).select("SO2_column_number_density").mean().rename('SO2')
    CH4 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CH4').filterDate(startDate, endDate).select("CH4_column_volume_mixing_ratio_dry_air").mean().rename('CH4')
    O3 = ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_O3').filterDate(startDate, endDate).select("O3_column_number_density").mean().rename('O3')

    print("Sentinel 5P layers prepared.")

except Exception as e:
    print(f"Error loading or processing Sentinel-5P ImageCollection: {e}")

Sentinel 5P layers prepared.


In [ ]:
# Vzorkovani a prevod do DataFrame
results = []

for image, name in zip([CO, HCHO, NO2, SO2, CH4, O3], ['CO', 'HCHO', 'NO2', 'SO2', 'CH4', 'O3']):
    samples = image.sampleRegions(
        collection=Body1400_ee,
        scale=1000,
        geometries=True
    )
    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=f'Sentinel_5P_sample_export_{name}',
        folder='GEE_output',
        fileNamePrefix=f'sentinel5P_samples_{name}',
        fileFormat='CSV'
    )
    task.start()
    results.append((task, name))

In [ ]:
# Kontrola uloh a nacteni dat
for task, name in results:
    print(f'Waiting for the export of {name} to complete...')
    while task.active():
        print(f'Exporting {name}...')
        time.sleep(30)

    if task.status()['state'] == 'COMPLETED':
        print(f'Export of {name} completed successfully.')
    else:
        print(f'Export of {name} failed:', task.status())

Waiting for the export of CO to complete...
Exporting CO...
Export of CO completed successfully.
Waiting for the export of HCHO to complete...
Export of HCHO completed successfully.
Waiting for the export of NO2 to complete...
Export of NO2 completed successfully.
Waiting for the export of SO2 to complete...
Export of SO2 completed successfully.
Waiting for the export of CH4 to complete...
Exporting CH4...
Export of CH4 completed successfully.
Waiting for the export of O3 to complete...
Export of O3 completed successfully.


In [ ]:
CO_file = '/content/drive4/MyDrive/DP/GEE_output/sentinel5P_samples_CO.csv'
HCHO_file = '/content/drive4/MyDrive/DP/GEE_output/sentinel5P_samples_HCHO.csv'
NO2_file = '/content/drive4/MyDrive/DP/GEE_output/sentinel5P_samples_NO2.csv'
SO2_file = '/content/drive4/MyDrive/DP/GEE_output/sentinel5P_samples_SO2.csv'
CH4_file = '/content/drive4/MyDrive/DP/GEE_output/sentinel5P_samples_CH4.csv'
O3_file = '/content/drive4/MyDrive/DP/GEE_output/sentinel5P_samples_O3.csv'

In [ ]:
dataCO = pd.read_csv(CO_file)
dataHCHO = pd.read_csv(HCHO_file)
dataNO2 = pd.read_csv(NO2_file)
dataSO2 = pd.read_csv(SO2_file)
dataCH4 = pd.read_csv(CH4_file)
dataO3 = pd.read_csv(O3_file)

In [ ]:
print(dataCO.head())

  system:index        CO  ID  LDEN   MERGE_SRC    Shape_Leng  \
0          0_0  0.030732   1  60.0  aglomerace  2.618360e+06   
1          1_0  0.030479   2  65.0     silnice  1.327160e+06   
2          2_0  0.030413   3  80.0     silnice  7.238570e+05   
3          3_0  0.029961   4  70.0   zeleznice  8.856371e+05   
4          4_0  0.029787   5  65.0   zeleznice  1.107023e+06   

          noiseLevel                                               .geo  
0           Lden5559  {"geodesic":false,"type":"Point","coordinates"...  
1           Lden6064  {"geodesic":false,"type":"Point","coordinates"...  
2  LdenGreaterThan75  {"geodesic":false,"type":"Point","coordinates"...  
3           Lden6569  {"geodesic":false,"type":"Point","coordinates"...  
4           Lden6064  {"geodesic":false,"type":"Point","coordinates"...  


## 2.6 VIIRS

#### Světelné znečištění

##### Stažení dat z EOG

###### VNL v2.2 pro rok 2022

In [ ]:
# celorovni prumer
#url = 'https://eogdata.mines.edu/nighttime_light/annual/v22/2022/VNL_v22_npp-j01_2022_global_vcmslcfg_c202303062300.average.dat.tif.gz'

In [ ]:
gz_file = os.path.join(shp_dir, 'VNL_v22_npp-j01_2022_global_vcmslcfg_c202303062300.average.dat.tif.gz')

###### SVDNB v10 pro 02/2022

In [ ]:
# unor 2022
#url = 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2022/202202/vcmslcfg/SVDNB_npp_20220201-20220228_global_vcmslcfg_v10_c202203052300.avg_rade9h.tif.gz'

In [ ]:
 gz_file = os.path.join(shp_dir, 'SVDNB_npp_20220201-20220228_global_vcmslcfg_v10_c202203052300.avg_rade9h.tif.gz')

###### SVDNB v10 pro 05/2022

In [ ]:
# kveten 2022
#url = 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2022/202205/vcmslcfg/SVDNB_npp_20220501-20220531_global_vcmslcfg_v10_c202206071200.avg_rade9h.tif.gz'

In [ ]:
gz_file = os.path.join(shp_dir, 'SVDNB_npp_20220501-20220531_global_vcmslcfg_v10_c202206071200.avg_rade9h.tif.gz')

###### SVDNB v10 pro 08/2022

In [ ]:
# srpen 2022
#url = 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2022/202208/NOAA-20/vcmslcfg/SVDNB_j01_20220801-20220831_global_vcmslcfg_v10_c202209231200.avg_rade9h.tif.gz'

In [ ]:
gz_file = os.path.join(shp_dir, 'SVDNB_j01_20220801-20220831_global_vcmslcfg_v10_c202209231200.avg_rade9h.tif.gz')

###### SVDNB v10 pro 11/2022

In [ ]:
# listopad 2022
#url = 'https://eogdata.mines.edu/nighttime_light/monthly_notile/v10/2022/202211/vcmslcfg/SVDNB_npp_20221101-20221130_global_vcmslcfg_v10_c202212131200.avg_rade9h.tif.gz'

In [ ]:
gz_file = os.path.join(shp_dir, 'SVDNB_npp_20221101-20221130_global_vcmslcfg_v10_c202212131200.avg_rade9h.tif.gz')

##### Rozbalení dat

In [ ]:
#response = requests.get(url, stream=True)
#if response.status_code != 200:
#    raise Exception(f"Failed to download file. Status code: {response.status_code}")

In [ ]:
#if not os.path.exists(shp_dir): # kontrola vystupniho adresare
#    os.makedirs(shp_dir)

#with open(gz_file, 'wb') as file:
#    for chunk in response.iter_content(chunk_size=1024):
#        if chunk:
#            file.write(chunk)

#print(f"File {gz_file} downloaded successfully.")

File /content/drive4/MyDrive/DP/SVDNB_npp_20220201-20220228_global_vcmslcfg_v10_c202203052300.avg_rade9h.tif.gz downloaded successfully.


In [ ]:
#tif_file = gz_file.replace('.gz', '')

In [ ]:
#import gzip, shutil, os

#try:
#    with gzip.open(gz_file, 'rb') as f_in:
#        with open(tif_file, 'wb') as f_out:
#            shutil.copyfileobj(f_in, f_out)
 #   print(f"File was extracted as {tif_file}.")
#except Exception as e:
#    print(f"Failed to extract file: {e}")


Failed to extract file: Not a gzipped file (b'<!')


##### Práce s daty

In [ ]:
#tiff_path = '/content/drive4/MyDrive/DP/VNL_v22_npp-j01_2022_global_vcmslcfg_c202303062300.average.dat.tif'

In [ ]:
tiff_path = os.path.splitext(gz_file)[0]

In [ ]:
with rasterio.open(tiff_path) as src:
    raster_crs = src.crs
    print("CRS of the raster file:", raster_crs)

print("CRS of the shapefile layer:", gdfCR.crs)

if raster_crs != gdfCR.crs:
    gdfCR = gdfCR.to_crs(raster_crs)
    print("Shapefile layer reprojected to match raster CRS.")


CRS of the raster file: EPSG:4326
CRS of the shapefile layer: EPSG:4326


In [ ]:
from osgeo import gdal
bbox = gdfCR.total_bounds # ohranicujici bbox
gdal.Warp('/content/drive4/MyDrive/DP/SVDNB_v10_11_2022_CR_vcmslcfg_avg.tif', tiff_path, outputBounds=bbox)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f56d9000db0> >

In [ ]:
os.listdir('/content/drive4/MyDrive/DP')

['oblast_hlukData.prj',
 'oblast_hlukData.cpg',
 'oblast_hlukData.shp',
 'oblast_hlukData.sbn',
 'oblast_hlukData.shx',
 'oblast_hlukData.dbf',
 'oblast_hlukData.shp.xml',
 'oblast_hlukData.sbx',
 '.ipynb_checkpoints',
 'VNL_v22_npp-j01_2022_global_vcmslcfg_c202303062300.average.dat.tif',
 'outputVRT.vrt',
 'STATY_P.shx',
 'STATY_P.cpg',
 'STATY_P.prj',
 'STATY_P.dbf',
 'STATY_P.shp',
 'SHM_Body1400.shx',
 'SHM_Body1400.cpg',
 'SHM_Body1400.sbx',
 'SHM_Body1400.shp',
 'SHM_Body1400.shp.xml',
 'SHM_Body1400.prj',
 'SHM_Body1400.dbf',
 'SHM_Body1400.sbn',
 'SHM_Body1400_SJ.cpg',
 'SHM_Body1400_SJ.sbn',
 'SHM_Body1400_SJ.sbx',
 'SHM_Body1400_SJ.dbf',
 'SHM_Body1400_SJ.prj',
 'SHM_Body1400_SJ.shp',
 'SHM_Body1400_SJ.shp.xml',
 'SHM_Body1400_SJ.shx',
 'GEE_output_old',
 'SHM_Body1400_1000m.shp.xml',
 'SHM_Body1400_1000m.cpg',
 'SHM_Body1400_1000m.dbf',
 'SHM_Body1400_1000m.shp',
 'SHM_Body1400_1000m.sbn',
 'SHM_Body1400_1000m.shx',
 'SHM_Body1400_1000m.prj',
 'SHM_Body1400_1000m.sbx',
 'GEE

In [ ]:
# doplneni sloupce cisloRadku
gdfBody['ID'] = gdfBody.index.astype(str)
print(gdfBody[['ID']].head())

  ID
0  0
1  1
2  2
3  3
4  4


Kontrola CSR + převedení na stejný CSR

In [ ]:
print("CRS of point layer:", gdfBody.crs)
with rasterio.open(tiff_path) as src:
    print("CRS of TIFF file:", src.crs)

CRS of point layer: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
CRS of TIFF file: EPSG:4326


In [ ]:
# Prevod na stejny CRS
with rasterio.open(tiff_path) as src:
    tiff_crs = src.crs

gdfBody = gdfBody.to_crs(tiff_crs)

In [ ]:
# kontrola
#with rasterio.open(tiff_path) as src:
#    print("Bounds of TIFF:", src.bounds)
#    print("Bounds of point layer:", gdfBody.total_bounds)

#with rasterio.open(tiff_path) as src:
 #   data = src.read(1)  # 1. pasmo
 #   print("Unique values in TIFF:", np.unique(data))

In [ ]:
from rasterio.mask import mask
with rasterio.open(tiff_path) as src:
    # Extrahovani souradnic bodu
    coords = [(x, y) for x, y in zip(gdfBody.geometry.x, gdfBody.geometry.y)]
    values = [val[0] for val in src.sample(coords)]  # Hodnot rastru jednotlive body
    print(f"Extracted raster values: {values[:5]}")

# Přidání extrahovaných hodnot do původní bodové vrstvy
gdfBody['raster_value'] = values
print(gdfBody.head())

Extracted raster values: [5.77, 2.44, 1.04, 1.55, 1.25]
  ID   LDEN         noiseLevel   MERGE_SRC    Shape_Leng  \
0  0  60.00           Lden5559  aglomerace  2.618360e+06   
1  1  65.00           Lden6064     silnice  1.327160e+06   
2  2  80.00  LdenGreaterThan75     silnice  7.238570e+05   
3  3  70.00           Lden6569   zeleznice  8.856371e+05   
4  4  65.00           Lden6064   zeleznice  1.107023e+06   

                    geometry  raster_value  
0  POINT (16.62017 49.12896)          5.77  
1  POINT (13.33214 49.67493)          2.44  
2  POINT (13.20077 49.70329)          1.04  
3  POINT (16.48606 49.78299)          1.55  
4  POINT (16.57605 49.89006)          1.25  


In [ ]:
print(gdfBody)

        ID   LDEN         noiseLevel   MERGE_SRC    Shape_Leng  \
0        0  60.00           Lden5559  aglomerace  2.618360e+06   
1        1  65.00           Lden6064     silnice  1.327160e+06   
2        2  80.00  LdenGreaterThan75     silnice  7.238570e+05   
3        3  70.00           Lden6569   zeleznice  8.856371e+05   
4        4  65.00           Lden6064   zeleznice  1.107023e+06   
...    ...    ...                ...         ...           ...   
1395  1395  75.00           Lden7074   zeleznice  1.043129e+06   
1396  1396  55.00           Lden5054  aglomerace  9.099600e+05   
1397  1397  55.00           Lden5054     silnice  7.512108e+06   
1398  1398  60.00           Lden5559     silnice  1.537878e+06   
1399  1399  55.00           Lden5054     silnice  2.570603e+06   

                       geometry  raster_value  
0     POINT (16.62017 49.12896)          5.77  
1     POINT (13.33214 49.67493)          2.44  
2     POINT (13.20077 49.70329)          1.04  
3     POINT (16

In [ ]:
print(gdfBody.columns)

Index(['ID', 'LDEN', 'noiseLevel', 'MERGE_SRC', 'Shape_Leng', 'geometry',
       'raster_value'],
      dtype='object')


In [ ]:
lightPoll_path = '/content/drive4/MyDrive/DP/GEE_output/viirs_11_2022.csv'
gdfBody[['ID', 'raster_value']].to_csv(lightPoll_path, index=False)
print(f"CSV file saved successfully at {lightPoll_path}")

CSV file saved successfully at /content/drive4/MyDrive/DP/GEE_output/viirs_11_2022.csv
